<a href="https://colab.research.google.com/github/Yash-Kamtekar/Approximate-nearest-neighbor/blob/main/Approximate_nearest_neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing all the necessary libraries

In [12]:
import numpy as np
import pickle
import pandas as pd
import time

Importing the lightfm library to import the dataset.
First we need to install the library.

In [13]:
pip install lightfm

In [14]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

importing the movielens dataset and getting the train and test data.

In [15]:
movie_lens = fetch_movielens()

train = movie_lens['train']
test = movie_lens['test']

There are 2 models that lightfm uses and we will use both to see which one is better.

1. let us train the model using Bayesian Personalised Ranking (bpr) and look at its accuracy.

In [16]:
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

bpr_precision_train = precision_at_k(model, train, k=10).mean()
bpr_precision_test = precision_at_k(model, test, k=10, train_interactions=train).mean()

bpr_auc_train = auc_score(model, train).mean()
bpr_auc_test = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (bpr_precision_train, bpr_precision_test))
print('AUC: train %.2f, test %.2f.' % (bpr_auc_train, bpr_auc_test))

Precision: train 0.58, test 0.19.
AUC: train 0.89, test 0.88.


2. Now, let us train the model using Weighted Approximate-Rank Pairwise (warp) and look at its accuracy.

In [17]:
model = LightFM(learning_rate=0.05, loss='warp')
model.fit_partial(train, epochs=10)

warp_precision_train = precision_at_k(model, train, k=10).mean()
warp_precision_test = precision_at_k(model, test, k=10, train_interactions=train).mean()

warp_auc_train = auc_score(model, train).mean()
warp_auc_test = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (warp_precision_train, warp_precision_test))
print('AUC: train %.2f, test %.2f.' % (warp_auc_train, warp_auc_test))

Precision: train 0.59, test 0.22.
AUC: train 0.93, test 0.93.


we can clearly get slightly higher precision in warp than bpr.

In [18]:
item_vectors = movie_lens['item_features'] * model.item_embeddings
item_vectors

array([[ 0.63929015, -0.7866718 ,  0.43742874, ...,  0.35012692,
        -0.72927636, -0.6804868 ],
       [-0.4790248 , -0.6651845 ,  0.5624019 , ..., -0.1497676 ,
        -0.31768283, -0.23793122],
       [ 0.28179494, -0.3991553 ,  0.14451884, ..., -0.57597595,
        -0.39641306,  0.02813605],
       ...,
       [-0.093698  ,  0.26761952, -0.44490165, ..., -0.2473825 ,
         0.37607962,  0.52969223],
       [-0.21398911,  0.25069958, -0.33324805, ..., -0.2502132 ,
         0.325372  ,  0.39941004],
       [-0.12749988,  0.19779742, -0.3461629 , ..., -0.42742547,
         0.13021772,  0.2841133 ]], dtype=float32)

let us store this data in a variable.
and also save it in a pickle file.

In [19]:
with open('movie_lens.pickle', 'wb') as f:
    pickle.dump({"name": movie_lens['item_feature_labels'], "vector": item_vectors}, f)

data = ({"name": movie_lens['item_feature_labels'], "vector": item_vectors})
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[ 0.63929015, -0.7866718 ,  0.43742874, ...,  0.35012692,
         -0.72927636, -0.6804868 ],
        [-0.4790248 , -0.6651845 ,  0.5624019 , ..., -0.1497676 ,
         -0.31768283, -0.23793122],
        [ 0.28179494, -0.3991553 ,  0.14451884, ..., -0.57597595,
         -0.39641306,  0.02813605],
        ...,
        [-0.093698  ,  0.26761952, -0.44490165, ..., -0.2473825 ,
          0.37607962,  0.52969223],
        [-0.21398911,  0.25069958, -0.33324805, ..., -0.2502132 ,
          0.325372  ,  0.39941004],
        [-0.12749988,  0.19779742, -0.3461629 , ..., -0.42742547,
          0.13021772,  0.2841133 ]], dtype=float32)}

# **Locality Sensitive Hashing**

lets install faiss and import it.

In [20]:
!pip install faiss-gpu
import faiss

Creating index class

In [21]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
          
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

Let us calculate the time taken for the model to train using LSH.

In [22]:
start_time = time.time()

index = LSHIndex(data["vector"], data["name"])
index.build()

movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))

end_time = time.time()

total_time = end_time - start_time

print("LSH took {total_time} seconds.".format(total_time=total_time))

print(f"The most similar movies to {movie_name} are:\n* {simlar_movie_questions}")

LSH took 0.003084421157836914 seconds.
The most similar movies to Nightmare Before Christmas, The (1993) are:
* Maverick (1994)
* Muppet Treasure Island (1996)
* Jurassic Park (1993)
* Lion King, The (1994)
* Sleepless in Seattle (1993)
* Nightmare Before Christmas, The (1993)
* Aladdin (1992)
* Dances with Wolves (1990)
* Mask, The (1994)
* While You Were Sleeping (1995)


# **Exhaustive Search**

Creating index class

In [23]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

Let us calculate the time taken for the model to train using exhaustive search.

In [24]:
start_time = time.time()

index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))

end_time = time.time()

total_time = end_time - start_time

print("Exhaustive search took {total_time} seconds.".format(total_time=total_time))

print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

Exhaustive search took 0.0008196830749511719 seconds.
The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Man Without a Face, The (1993)
* Pink Floyd - The Wall (1982)
* Maverick (1994)
* Cinderella (1950)
* Client, The (1994)
* Grease (1978)
* Sneakers (1992)
* Tombstone (1993)


# **Product Quantization**

Creating index class

In [25]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimention, number_of_partition, search_in_x_partitions, subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)


    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

Let us calculate the time taken for the model to train using product quantization.

In [26]:
start_time = time.time()

index = IVPQIndex(data["vector"], data["name"])
index.build()

movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))

end_time = time.time()

total_time = end_time - start_time

print("Product Quantization took {total_time} seconds.".format(total_time=total_time))

print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

Product Quantization took 0.10459399223327637 seconds.
The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Cinderella (1950)
* Maverick (1994)
* Man Without a Face, The (1993)
* So I Married an Axe Murderer (1993)
* Pink Floyd - The Wall (1982)
* Tombstone (1993)
* Grease (1978)
* Batman (1989)


# **Trees and Graph**

lets install annoy and import it.

In [27]:
!pip install annoy
import annoy

     |████████████████████████████████| 646 kB 35.8 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391677 sha256=03c71a195149acaf6408374adfb9cb995d17bae6510e1e3ac48db69d245c534c
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


Creating index class

In [28]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)

    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

Let us calculate the time taken for the model to train using trees & graph.

In [29]:
start_time = time.time()

index = AnnoyIndex(data["vector"], data["name"])
index.build()

movie_vector, movie_name = data['vector'][90], data['name'][90]
similar_movie_questions = '\n* '.join(index.query(movie_vector))

end_time = time.time()

total_time = end_time - start_time

print("Trees and Graph took {total_time} seconds.".format(total_time=total_time))

print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

Trees and Graph took 0.018578529357910156 seconds.
The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mask, The (1994)
* Cinderella (1950)
* Maverick (1994)
* Man Without a Face, The (1993)
* So I Married an Axe Murderer (1993)
* Pink Floyd - The Wall (1982)
* Tombstone (1993)
* Grease (1978)
* Batman (1989)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


# **Hierarchical Navigable Small World Algorithm**

lets install nmslib and import it.

In [30]:
!pip install nmslib
import nmslib

     |████████████████████████████████| 13.5 MB 35.9 MB/s 
     |████████████████████████████████| 188 kB 69.7 MB/s 


Creating index class

In [31]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})

    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

Let us calculate the time taken for the model to train using HNSW.

In [32]:
start_time = time.time()

index = HNSWIndex(data["vector"], data["name"])
index.build()

movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))

end_time = time.time()

total_time = end_time - start_time

print("HNSW took {total_time} seconds.".format(total_time=total_time))

print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

HNSW took 0.23832941055297852 seconds.
The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Batman (1989)
* Client, The (1994)
* Grease (1978)
* Mask, The (1994)
* Nell (1994)
* Man Without a Face, The (1993)
* Maverick (1994)
* Cinderella (1950)
* Sleepless in Seattle (1993)
